In [ ]:
!pip install --upgrade transformers bitsandbytes peft accelerate datasets trl

: 

In [2]:
!nvidia-smi


Mon Mar 24 12:17:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
  Attempting uninstall: trl
    Found existing installation: trl 0.16.0
    Uninstalling trl-0.16.0:
      Successfully uninstalled trl-0.16.0
ERROR: pip's dependency resolver does not currently t

In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00


In [27]:
#@title First test mTGE family model without semantics understanding
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
import numpy as np
import evaluate

# 1. Load tokenizer and classification head model
model_name = "Alibaba-NLP/gte-multilingual-mlm-base"
num_labels = 7  # binary classification
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels,trust_remote_code=True)

# 2. Load a classification dataset (IMDB as example)
dataset = load_dataset("tridm/UIT-VSMEC")

# 2. Fixed emotion-to-id mapping
label2id = {
    'Anger': 0,
    'Disgust': 1,
    'Enjoyment': 2,
    'Fear': 3,
    'Other': 4,
    'Sadness': 5,
    'Surprise': 6
}
id2label = {v: k for k, v in label2id.items()}
label2id = {label: idx for idx, label in enumerate(sorted(label_list))}
print("Label mapping:", label2id)

# 2. Map Emotion strings to integers
def encode_labels(example):
    example["labels"] = label2id[example["Emotion"]]
    return example

dataset = dataset.map(encode_labels)
# 3. Preprocess (tokenize text)
def preprocess_function(examples):
    return tokenizer(examples["Sentence"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]
train_subset = train_dataset.shuffle(seed=42).select(range(500))
test_subset = test_dataset.shuffle(seed=42).select(range(200))
# 5. Load metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

# 6. TrainingArguments
training_args = TrainingArguments(
    output_dir="./gte-seqcls-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    push_to_hub=False,
    report_to="none",
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset, # use subset for speed
    eval_dataset=test_subset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 8. Train
trainer.train()

# 9. Evaluate
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# 10. Save the final model
trainer.save_model("./gte-seqcls-finetuned")
tokenizer.save_pretrained("./gte-seqcls-finetuned")


Some weights of NewForSequenceClassification were not initialized from the model checkpoint at Alibaba-NLP/gte-multilingual-mlm-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'new.pooler.dense.bias', 'new.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Label mapping: {'Anger': 0, 'Disgust': 1, 'Enjoyment': 2, 'Fear': 3, 'Other': 4, 'Sadness': 5, 'Surprise': 6}


Map:   0%|          | 0/5548 [00:00<?, ? examples/s]

Map:   0%|          | 0/686 [00:00<?, ? examples/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-27-f5b19c69f0c9>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.765010,0.265000
2,1.827200,1.689465,0.360000
3,1.827200,1.673695,0.350000


Evaluation Results: {'eval_loss': 1.6736950874328613, 'eval_accuracy': 0.35, 'eval_runtime': 7.3274, 'eval_samples_per_second': 27.295, 'eval_steps_per_second': 3.412, 'epoch': 3.0}


('./gte-seqcls-finetuned/tokenizer_config.json',
 './gte-seqcls-finetuned/special_tokens_map.json',
 './gte-seqcls-finetuned/tokenizer.json')

In [28]:
#@title NO classification, embedding + similarity NO finetuning
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import torch
import torch.nn.functional as F
from tqdm import tqdm

# 1. Load the GTE model for embeddings
model_name = "Alibaba-NLP/gte-multilingual-mlm-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model.eval()

# 2. Load dataset
dataset = load_dataset("tridm/UIT-VSMEC")
test_data = dataset["test"].shuffle(seed=42).select(range(200))

# 3. Define fixed label list and mapping
label_list = ['Anger', 'Disgust', 'Enjoyment', 'Fear', 'Other', 'Sadness', 'Surprise']

# 4. Get embedding of a sentence (uses [CLS] token)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0]  # [CLS] token
        return F.normalize(embeddings, dim=-1)

# 5. Precompute embeddings for all candidate labels
label_embeddings = torch.stack([get_embedding(label).squeeze(0) for label in label_list])

# 6. Predict and evaluate
correct = 0
all_preds = []
all_labels = []

for example in tqdm(test_data):
    sentence = example["Sentence"]
    true_label = example["Emotion"]

    sentence_emb = get_embedding(sentence)
    similarities = F.cosine_similarity(sentence_emb, label_embeddings)
    pred_idx = torch.argmax(similarities).item()
    pred_label = label_list[pred_idx]

    all_preds.append(pred_label)
    all_labels.append(true_label)

    if pred_label == true_label:
        correct += 1

# 7. Compute accuracy
accuracy = correct / len(test_data)
print(f"\nAccuracy: {accuracy:.4f}")


100%|██████████| 200/200 [00:29<00:00,  6.70it/s]


Accuracy: 0.1850


In [30]:
#@title Constractive training with full string target variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from torch.optim import AdamW
from datasets import load_dataset
from tqdm import tqdm
import random

# ========== 1. Load model and tokenizer ==========
model_name = "Alibaba-NLP/gte-multilingual-mlm-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ========== 2. Load dataset ==========
dataset = load_dataset("tridm/UIT-VSMEC")
train_data = dataset["train"].shuffle(seed=42).select(range(1000))  # for speed
eval_data = dataset["test"].shuffle(seed=42).select(range(200))

# ========== 3. Format for contrastive training ==========
label_list = ['Anger', 'Disgust', 'Enjoyment', 'Fear', 'Other', 'Sadness', 'Surprise']

class ContrastiveDataset(Dataset):
    def __init__(self, data, tokenizer, label_list, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.labels = label_list
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]["Sentence"]
        emotion = self.data[idx]["Emotion"]

        # Positive: (sentence, correct label)
        input_enc = self.tokenizer(sentence, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        label_enc = self.tokenizer(emotion, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": input_enc["input_ids"].squeeze(0),
            "attention_mask": input_enc["attention_mask"].squeeze(0),
            "label_ids": label_enc["input_ids"].squeeze(0),
            "label_attention_mask": label_enc["attention_mask"].squeeze(0),
            "emotion": emotion
        }

train_dataset = ContrastiveDataset(train_data, tokenizer, label_list)
eval_dataset = ContrastiveDataset(eval_data, tokenizer, label_list)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=1)

# ========== 4. Contrastive loss function ==========
def cosine_sim(a, b):
    a = F.normalize(a, dim=-1)
    b = F.normalize(b, dim=-1)
    return torch.matmul(a, b.T)

def contrastive_loss(sent_embs, label_embs):
    sim = cosine_sim(sent_embs, label_embs)  # [B, B]
    labels = torch.arange(sim.size(0)).to(sim.device)
    return F.cross_entropy(sim, labels)

# ========== 5. Optimizer ==========
optimizer = AdamW(model.parameters(), lr=2e-5)

# ========== 6. Training Loop ==========
for epoch in range(3):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for batch in pbar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label_ids = batch["label_ids"].to(device)
        label_attention_mask = batch["label_attention_mask"].to(device)

        # Get embeddings ([CLS] token)
        sent_out = model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
        label_out = model(input_ids=label_ids, attention_mask=label_attention_mask).last_hidden_state[:, 0]

        loss = contrastive_loss(sent_out, label_out)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} - Avg loss: {total_loss / len(train_loader):.4f}")

# ========== 7. Evaluation ==========
model.eval()
correct = 0

# Precompute label embeddings
with torch.no_grad():
    label_embs = []
    for label in label_list:
        tokens = tokenizer(label, return_tensors="pt", truncation=True, padding=True).to(device)
        label_emb = model(**tokens).last_hidden_state[:, 0]
        label_embs.append(F.normalize(label_emb, dim=-1))
    label_embs = torch.cat(label_embs, dim=0)  # [num_labels, hidden]

for batch in tqdm(eval_loader, desc="Evaluating"):
    input_ids = batch["input_ids"].unsqueeze(0).to(device)
    attention_mask = batch["attention_mask"].unsqueeze(0).to(device)
    true_emotion = batch["emotion"]

    with torch.no_grad():
        sentence_emb = model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
        sentence_emb = F.normalize(sentence_emb, dim=-1)
        sims = cosine_sim(sentence_emb, label_embs)  # [1, num_labels]
        pred_idx = torch.argmax(sims, dim=1).item()
        pred_label = label_list[pred_idx]

    if pred_label == true_emotion[0]:
        correct += 1

accuracy = correct / len(eval_dataset)
print(f"\nTop-1 Accuracy: {accuracy:.4f}")


Epoch 1:   3%|▎         | 2/63 [00:03<01:45,  1.73s/it, loss=2.81]


OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 316.12 MiB is free. Process 4873 has 14.43 GiB memory in use. Of the allocated memory 10.35 GiB is allocated by PyTorch, and 3.95 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)